PORTFEUILLE MIN VARIANCE :

In [72]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Parameters:
period = '3y'
# List of stocks
tickers = ["AAPL", "MSFT", "GOOGL"]

# Download adjusted historical data for the past 5 years
def download_stock_data(ticker_list):
    data = {}
    for ticker in ticker_list:
        stock = yf.Ticker(ticker)
        data[ticker] = stock.history(period="5y")['Close']
    return pd.DataFrame(data)

# Calculate daily returns
def calculate_daily_returns(dataframe):
    return dataframe.pct_change()

# Objective function to minimize portfolio variance
def portfolio_variance(weights, covariance_matrix):
    return np.dot(weights.T, np.dot(covariance_matrix, weights))

# Constraint function that the sum of weights must equal 1
def check_sum(weights):
    return np.sum(weights) - 1

# Download data for the stocks
stock_data = download_stock_data(tickers)

# Calculate daily returns
daily_returns = calculate_daily_returns(stock_data)

# Calculate annualized covariance matrix
covariance_matrix = daily_returns.cov() * 252

# Number of stocks in the portfolio
n = len(tickers)

# Constraints and bounds for optimization
constraints = ({'type': 'eq', 'fun': check_sum})
bounds = tuple((0, 1) for asset in range(n))

# Equal initial weights for the starting point of optimization
initial_weights = np.array([1/n] * n)

# Optimization to minimize portfolio variance
optimal_weights_result = minimize(portfolio_variance, initial_weights, args=(covariance_matrix,), method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = optimal_weights_result.x

# Calculate the variance of the optimized portfolio
optimized_portfolio_variance = portfolio_variance(optimal_weights, covariance_matrix)

# Calculate the expected return of the optimized portfolio
annual_returns = daily_returns.mean() * 252
optimized_portfolio_return = np.dot(optimal_weights, annual_returns)

# Convert weights and returns to percentages
optimal_weights_percent = optimal_weights * 100
optimized_portfolio_variance_percent = optimized_portfolio_variance * 100
optimized_portfolio_return_percent = optimized_portfolio_return * 100

# Create a DataFrame for optimized weights with names for readability
optimal_weights_df = pd.DataFrame(data={'Ticker': tickers, 'Weight (%)': optimal_weights_percent})

# Display the results
print("Optimal Weights per Stock:")
print(optimal_weights_df)
print("\nOptimized Portfolio Variance (%):", optimized_portfolio_variance_percent)
print("Expected Return of the Optimized Portfolio (%):", optimized_portfolio_return_percent)
print("Sharpe Ratio", optimized_portfolio_return/np.sqrt(optimized_portfolio_variance))


Optimal Weights per Stock:
  Ticker  Weight (%)
0   AAPL   29.836580
1   MSFT   37.478117
2  GOOGL   32.685303

Optimized Portfolio Variance (%): 8.030917402685416
Expected Return of the Optimized Portfolio (%): 31.67826077366969
Sharpe Ratio 1.1178376935213716
